In [ ]:
!nvidia-smi

In [ ]:
#%pip install --extra-index-url=https://pypi.nvidia.com cudf-cu12
#%pip install cucim-cu12 cupy-cuda12x

In [ ]:
%pip install fastparquet

In [1]:
#%load_ext cudf.pandas

# To desable GPU usage
#import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

#from cucim.skimage.exposure import rescale_intensity
import tensorflow as tf
#import cupy as cp
#import torch
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import seaborn as sns
import glob
from sklearn.model_selection import train_test_split
import gc

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # memory limit 16GB (16 * 1024 MB = 16384 MB) 
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=16384)] 
        )
        print("Set GPU memory limit to 16GB.")
    except RuntimeError as e:
        print("Error setting memory limit:", e)
else:
    print("No GPUs available.")

#print("Is torch using cuda? ",torch.cuda.is_available())
print("Is tensorflow using cuda? ",tf.test.is_built_with_cuda())
print("Is pandas using cuda? ",pd)


2024-11-17 00:06:01.489373: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-17 00:06:01.520803: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-17 00:06:01.530452: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-17 00:06:01.553410: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Set GPU memory limit to 16GB.
Is tensorflow using cuda?  True
Is pandas using cuda?  <module 'pandas' from '/usr/local/lib/python3.11/dist-packages/pandas/__init__.py'>


In [ ]:
name_mapping = [
    "box",
    "circularTorus",
    "cone",
    "coneOffset",
    "cylinder",
    "cylinderSlope",
    "dish",
    "mesh",
    "pyramid",
    "rectangularTorus",
    "sphere"
]

In [ ]:
def sort_by_number(texts:list[str]):
    def key(text:str):
        text = re.sub(r'.*photos_', '', text)
        text = re.sub(r'\.csv', '', text)
        text = re.sub(r'\D', '', text)
        return int(text)
    return sorted(texts, key=key)

In [22]:
!ls /home/workspace

tfrecord


In [12]:
base_path = '/home/workspace/geometry-classifier/data/'

def parse_tfrecord(example_proto):
    """Parse a single TFRecord example."""
    feature_description = {
        'name': tf.io.FixedLenFeature([], tf.string),
        'image': tf.io.FixedLenFeature([], tf.string)
    }
    parsed_example = tf.io.parse_single_example(example_proto, feature_description)
    name = tf.io.decode_raw(parsed_example['name'], tf.uint8)
    image = tf.io.decode_raw(parsed_example['image'], tf.uint8)
    image = tf.reshape(image, (224, 224, 1))  # Reshape image data to 224x224x3
    class_label = tf.cast(name[0], tf.int32)  # Use the first byte as a class label
    return image, class_label

def load_tfrecord(tfrecord_files):
    """Load TFRecords and prepare a dataset."""
    raw_dataset = tf.data.TFRecordDataset(tfrecord_files, compression_type="GZIP")
    parsed_dataset = raw_dataset.map(parse_tfrecord)
    return parsed_dataset

# Directory with TFRecords

files_for = lambda file_type: glob.glob(f"{base_path}tfrecord/tfrecord/{file_type}*.tfrecord.gz")

test_files = files_for("test")
validation_files = files_for("val")
train_files = files_for("train")

print(glob.glob(f"{base_path}tfrecord/tfrecord/*.tfrecord.gz"))

test = load_tfrecord(test_files)
test = test.batch(32).prefetch(tf.data.AUTOTUNE)

validation = load_tfrecord(validation_files)
validation = validation.batch(32).prefetch(tf.data.AUTOTUNE)

train = load_tfrecord(train_files)
train = train.batch(32).shuffle(1000).prefetch(tf.data.AUTOTUNE)



['/home/workspace/geometry-classifier/data/tfrecord/tfrecord/train462.tfrecord.gz', '/home/workspace/geometry-classifier/data/tfrecord/tfrecord/train153.tfrecord.gz', '/home/workspace/geometry-classifier/data/tfrecord/tfrecord/test54.tfrecord.gz', '/home/workspace/geometry-classifier/data/tfrecord/tfrecord/train314.tfrecord.gz', '/home/workspace/geometry-classifier/data/tfrecord/tfrecord/train374.tfrecord.gz', '/home/workspace/geometry-classifier/data/tfrecord/tfrecord/train259.tfrecord.gz', '/home/workspace/geometry-classifier/data/tfrecord/tfrecord/test17.tfrecord.gz', '/home/workspace/geometry-classifier/data/tfrecord/tfrecord/train204.tfrecord.gz', '/home/workspace/geometry-classifier/data/tfrecord/tfrecord/train487.tfrecord.gz', '/home/workspace/geometry-classifier/data/tfrecord/tfrecord/train68.tfrecord.gz', '/home/workspace/geometry-classifier/data/tfrecord/tfrecord/train230.tfrecord.gz', '/home/workspace/geometry-classifier/data/tfrecord/tfrecord/train466.tfrecord.gz', '/home/w

In [13]:
# 224 x 224

from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam

model = models.Sequential([
    layers.Conv2D(8, (3, 3), activation='relu', input_shape=(224,224,1)),
    layers.MaxPooling2D((2, 2)),
    

    layers.Conv2D(16, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    

    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    

    layers.Flatten(),
    
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax') 
])

model.compile(optimizer=Adam(learning_rate=1e-4, clipnorm=1.0),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
!df -h

In [14]:
checkpoint_callback = ModelCheckpoint(base_path+'model.keras',save_best_only=True, save_weights_only=False, mode='min', verbose=1)
early_stopping = EarlyStopping(
    monitor="val_loss",       # Monitor validation loss
    patience=5,               # Wait for 5 epochs without improvement
    restore_best_weights=True,  # Restore the best model weights
    verbose=1
)

epochs = 100
steps_in_total = (1024 * 812 * 60)//100

In [15]:

cnn = model.fit(train, epochs=epochs,callbacks=[
    early_stopping,
    checkpoint_callback
], validation_data=validation)

Epoch 1/100


2024-11-17 00:34:57.372450: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:3990: Filling up shuffle buffer (this may take a while): 939 of 1000
2024-11-17 00:34:58.029775: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


 15601/498892 ━━━━━━━━━━━━━━━━━━━━ 1:23:28 10ms/step - accuracy: 0.0909 - loss: nan

2024-11-17 00:37:42.658967: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 10297416704713115405
2024-11-17 00:37:42.659076: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 4758223705653005746
/usr/lib/python3.11/contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)



Epoch 1: val_loss did not improve from inf
498892/498892 ━━━━━━━━━━━━━━━━━━━━ 231s 431us/step - accuracy: 0.0909 - loss: nan - val_accuracy: 0.0909 - val_loss: nan
Epoch 2/100


2024-11-17 00:38:35.988796: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 10297416704713115405
2024-11-17 00:38:35.988898: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 4758223705653005746


 11264/498892 ━━━━━━━━━━━━━━━━━━━━ 1:15:40 9ms/step - accuracy: 0.0909 - loss: nan

KeyboardInterrupt: 

In [ ]:
test_loss, test_accuracy = model.evaluate(test)
test_loss, test_accuracy

2024-11-17 00:28:27.141636: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:4: Filling up shuffle buffer (this may take a while): 817 of 1000


     21/Unknown 12s 8ms/step - accuracy: 0.0928 - loss: nan

2024-11-17 00:28:29.059462: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


5184/5184 ━━━━━━━━━━━━━━━━━━━━ 47s 7ms/step - accuracy: 0.0910 - loss: nan


2024-11-17 00:29:04.266960: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 10297416704713115405
/usr/lib/python3.11/contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)
2024-11-17 00:29:04.267089: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 4758223705653005746


(nan, 0.09090470522642136)